# Vanilla Incremental Elicitation Procedure

### Import des bibiliotéhques 

In [65]:
import numpy as np
from mip import Model, xsum, BINARY, minimize, maximize

### 1. Fonction M_Omega

In [66]:
# Fonction pour calculer le modèle de décision avec les poids
def M_omega(x, omega):
    return np.dot(x, omega)

- Calcule l'évaluation d'une solution 'x' avec un vecteur de poids 'omega'.
    
    Args:
    - x (array-like): Vecteur représentant une solution avec ses scores pour chaque critère.
    - omega (array-like): Vecteur représentant les poids associés à chaque critère.
    
    Returns:
    - float: La somme pondérée des scores de 'x' selon les poids 'omega'.

### 2. Fonction Query

In [67]:
# Fonction pour poser une question au décideur
def Query(x, y):
    # Simule la préférence du décideur (choix aléatoire ici, à remplacer par une vraie préférence)
    print(f"Comparaison entre {x} et {y}")
    return x if np.random.rand() > 0.5 else y

 - Pose une question au décideur pour comparer deux solutions 'x' et 'y'.
    La réponse est simulée par un choix aléatoire.
    
    Args:
    - x (array-like): Première solution à comparer.
    - y (array-like): Seconde solution à comparer.
    
    Returns:
    - array-like: La solution préférée entre 'x' et 'y'.

#### Résolution du probléme linéaire pour calculer PMR en utilisant MIP

La fonction PMR calcule le regret maximum par paires entre deux solutions x et y, en tenant compte des poids admissibles définis dans 
Ω. Le regret est la différence entre les évaluations des deux solutions, maximisée pour le pire ensemble de poids possible dans le polytope Ω. 

- Modélisation du problème : On modélise cela comme un problème de programmation linéaire où on veut maximiser la différence entre l’évaluation de y et de x en fonction des poids w (qui sont les variables de décision dans le modèle).

- On introduit une variable w pour chaque critère (chaque dimension du problème). Ces variables représentent les poids wi associés à chaque critère, qui doivent être trouvés pour maximiser le regret. Ces poids doivent respecter des contraintes de normalisation (somme égale à 1) et les autres contraintes définies dans Ω.

- Ajout des contraintes de Ω: Ω est un polytope défini par un ensemble de contraintes linéaires. Chaque contrainte dans Ω a la forme d’une équation linéaire qui doit être respectée pour que les poids soient admissibles. Dans le code, on itère sur toutes les contraintes définies dans Ω et on les ajoute au modèle.

-L’objectif est de maximiser la différence entre l'évaluation de y et x, c'est-à-dire maximiser Mw(y)-Mw(x), où Mw(.) est l’évaluation pondérée d’une solution par rapport aux poids w. Cela revient à maximiser la somme pondérée des différences entre y[i] et x[i].

### 3. Fonction PMR

In [68]:
# Fonction pour calculer le regret maximum par paires en tenant compte des contraintes de m
def PMR(x, y, m):  
    m_crit = len(x)  # Nombre de critères

    # Variables de décision : poids w_i pour chaque critère
    w = [m.add_var(lb=0) for _ in range(m_crit)]
    
    # Contrainte : somme des poids w_i = 1 (normalisation des poids)
    m += xsum(w[i] for i in range(m_crit)) == 1

    # Objectif : maximiser M_omega(y) - M_omega(x)
    m.objective = maximize(xsum(w[i] * (y[i] - x[i]) for i in range(m_crit)))
    
    # Résoudre le problème
    m.optimize()
    
    # Obtenir le regret maximal (valeur de la fonction objectif)
    return m.objective_value

 - Calcule le regret maximum par paires (PMR) entre deux solutions 'x' et 'y'.
    Utilise la programmation linéaire pour maximiser le regret sous les contraintes définies dans 'Omega'.
    
    Args:
    - x (array-like): Première solution.
    - y (array-like): Seconde solution.
    - m (Model): Modèle MIP dans lequel les contraintes et l'objectif seront définis.
    
    Returns:
    - float: La valeur du regret maximum entre 'x' et 'y', optimisée avec le modèle MIP.

### 4. Fonction Update : Mise à jour du polytope Ω

La fonction Update a pour but de mettre à jour le polytope Ω, qui représente l'ensemble des poids possibles, en fonction des réponses du décideur à une question de préférence entre deux solutions x et y.

- Décision du décideur : Le décideur est interrogé pour savoir s'il préfère x ou y. En fonction de sa réponse, on sait que le modèle de décision sous-jacent doit respecter certaines relations linéaires entre les poids w:
    * Si x est préféré à y, cela signifie que la somme pondérée des critères pour x doit être plus grande ou égale à celle de y:         Mw(x) >= Mw(y).
    * Sinon, l'inverse.

- Ici, les coefficients de la contrainte sont la différence entre les vecteurs x et y. Le côté droit de l'inégalité est 0 car on compare directement les évaluations pondérées.

- Mise à jour du polytope Ω : Une nouvelle contrainte est ajoutée au polytope Ω à chaque nouvelle réponse. Cette contrainte réduit l'espace des poids admissibles en excluant ceux qui ne respectent pas la préférence du décideur. Cette mise à jour affine la connaissance des préférences du décideur.

In [69]:
# Fonction pour mettre à jour le polytope des poids possibles dans m
def Update(m, answer, x, y):
    # Créer une nouvelle contrainte basée sur la réponse du décideur
    if np.array_equal(answer, x):
        # x est préféré à y -> ajouter la contrainte M_omega(x) >= M_omega(y)
        new_constraint = xsum((x[i] - y[i]) * m.var_by_name(f"w[{i}]") for i in range(len(x))) >= 0
    else:
        # y est préféré à x -> ajouter la contrainte M_omega(y) >= M_omega(x)
        new_constraint = xsum((y[i] - x[i]) * m.var_by_name(f"w[{i}]") for i in range(len(x))) >= 0
    
    # Ajouter la nouvelle contrainte au modèle
    m += new_constraint
    
    return m

 - Met à jour le modèle MIP 'm' avec une nouvelle contrainte en fonction de la réponse du décideur.
    Si 'x' est préféré à 'y', on ajoute une contrainte qui reflète cette préférence, et inversement.

    Args:
    - m (Model): Modèle MIP à mettre à jour.
    - answer (array): L'alternative préférée par le décideur (résultat de la fonction 'Query').
    - x (array): Première alternative comparée.
    - y (array): Deuxième alternative comparée.

    Returns:
    - Model: Le modèle MIP mis à jour avec la nouvelle contrainte.

In [70]:
# Fonction pour calculer le regret maximum (MR)
def MR(x, X, m):
    return max(PMR(x, y, m) for y in X if not np.array_equal(x, y))



-  Calcule le regret maximum pour une alternative donnée 'x' par rapport à toutes les autres alternatives dans 'X'.

    Args:
    - x (array): L'alternative pour laquelle on veut calculer le regret.
    - X (array of arrays): Ensemble d'alternatives à comparer avec 'x'.
    - m (Model): Modèle MIP utilisé pour optimiser les regrets.

    Returns:
    - float: Le regret maximum pour 'x' par rapport aux autres alternatives dans 'X'.


In [71]:
# Fonction pour calculer le regret minimax (mMR) et obtenir les solutions x_star et y_star
def mMR(X, m):
    x_star = min(X, key=lambda x: MR(x, X, m))
    y_star = max(X, key=lambda y: PMR(x_star, y, m))
    max_regret = MR(x_star, X, m)
    return max_regret, x_star, y_star



 - Calcule le regret minimax, c'est-à-dire l'alternative qui minimise son regret maximum par rapport aux autres.

    Args:
    - X (array of arrays): Ensemble d'alternatives.
    - m (Model): Modèle MIP utilisé pour optimiser les regrets.

    Returns:
    - tuple: (max_regret, x_star, y_star)
        - max_regret (float): Le regret minimax pour 'x_star'.
        - x_star (array): L'alternative qui minimise le regret maximum.
        - y_star (array): L'alternative qui maximise le regret par rapport à 'x_star'.

In [72]:
# Algorithme d'élucidation incrémentale vanilla
def vanilla_incremental_elicitation(X, m, epsilon):
    max_regret, x_star, y_star = mMR(X, m)
    print(max_regret)
    
    while max_regret >= epsilon:
        print(f"Regret actuel: {max_regret}")
        print(f"Question: préférez-vous {x_star} ou {y_star} ?")
        
        # Poser la question au décideur
        answer = Query(x_star, y_star)
        
        # Mettre à jour Omega en fonction de la réponse
        Omega = Update(m, answer, x_star, y_star)
        
        # Recalculer le regret minimax
        max_regret, x_star, y_star = mMR(X, m)
    
    return x_star

- Algorithme d'élucidation incrémentale vanilla. Cherche l'alternative optimale en minimisant le regret maximum
    et en posant des questions au décideur jusqu'à ce que le regret soit inférieur à un seuil 'epsilon'.

    Args:
    - X (array of arrays): Ensemble d'alternatives à comparer.
    - m (Model): Modèle MIP utilisé pour optimiser les regrets.
    - epsilon (float): Seuil de regret maximal accepté pour arrêter l'algorithme.

    Returns:
    - array: L'alternative optimale 'x_star' trouvée à la fin de l'élucidation.

In [73]:
# Exemple d'utilisation de l'élucidation
def exemple_elicitation():
    """
    Exemple d'utilisation de l'algorithme d'élucidation incrémentale.
    Définit un ensemble de solutions et lance l'algorithme pour trouver la solution optimale.
    
    Returns:
    - None
    """
    # Ensemble des solutions (chaque solution a 2 critères)
    X = np.array([[0.5, 0.2], [0.7, 0.1], [0.6, 0.3]])
    
    # Initialisation du modèle d'optimisation
    m = Model()
    
    # Tolérance pour le regret minimax
    epsilon = 0.1

    # Lancer l'algorithme
    solution = vanilla_incremental_elicitation(X, m, epsilon)

    # Afficher la solution finale recommandée
    print(f"\nSolution finale recommandée: {solution}")

In [74]:
# Lancer l'exemple
exemple_elicitation()

Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 0 (-1) rows, 0 (-2) columns and 0 (-2) elements
Clp0000I Optimal - objective value 0.2
Coin0511I After Postsolve, objective 0.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.2 - 0 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.2
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Opt